<a href="https://colab.research.google.com/github/poddubnyoleg/nondualbot/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

https://stackoverflow.com/questions/48760472/how-to-use-the-keras-model-to-forecast-for-future-dates-or-events

https://github.com/danmoller/TestRepo/blob/master/TestBookLSTM.ipynb

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/6/66/Nisargadatta_Maharaj.jpg/220px-Nisargadatta_Maharaj.jpg)



```
“Wisdom tells me I am nothing, love tells me I am everything. Between the two, my life flows.” 
― Sri Nisargadatta Maharaj```



---

I got idea of this project while reading [famous article of Andrey Karpathy about RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and thinking about if i can do something useful with generative ability of RNN. As I need to use neural nets in my [other project](https://github.com/poddubnyoleg/eeg_neurofeedback) with EEG data processing, it was fun to get NN experince in this fun project.


Core idea of project is to create a chat bot, that can keep any discussion and provide user with some useful (sometimes :) knowledge. I pick Sri Nisargadatta Maharaj as a knowledge carrier, whom i love for beutiful non-dual poetry, and his most famous book 'I am that' as data source for training.

In this notebook i guide you with the process of creating response engine, that can be based on your favourite text corpus.

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import keras.backend as K
import sys
from matplotlib import pyplot as plt

Using TensorFlow backend.


## Load text data
I use text file from google drive here. All preprocessing is considered to be done before

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file = open('/content/drive/My Drive/nondualbot/i_am_that.txt', 'r') 
data = file.read() 

In [5]:
data[:100]

'That which permeates all, which nothing transcends and which, like the universal space around us, fi'

In [0]:
chars = sorted(list(set(data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
n_chars = len(data)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  988168
Total Vocab:  55


For each chunk of data [seq_length] we take one-hot-encoded dataX array and dataY array (same but shifted +1)

In [106]:
seq_length = 100
batch_size = 100


K.clear_session()

model = Sequential()
model.add(LSTM(500, input_shape=(None, n_vocab), return_sequences=True, stateful=False))
#model.add(Dropout(0.5))
model.add(LSTM(500, input_shape=(None, n_vocab), return_sequences=True, stateful=False))
#model.add(Dropout(0.5))
model.add(LSTM(500, input_shape=(None, n_vocab), return_sequences=True, stateful=False))
#model.add(Dropout(0.5))

model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 500)         1112000   
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 500)         2002000   
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 500)         2002000   
_________________________________________________________________
dense_1 (Dense)              (None, None, 55)          27555     
Total params: 5,143,555
Trainable params: 5,143,555
Non-trainable params: 0
_________________________________________________________________


In [103]:
filename = "/content/drive/My Drive/nondualbot/best_current_weights_2.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

ValueError: ignored

In [0]:
filepath="/content/drive/My Drive/nondualbot/best_current_weights_2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
num_data = [char_to_int[char] for char in data]

**train with batch generator with fixed or variable batch sequense size**
* Inner state doesnt share between samples inside of one batch, but can share between batches, if stateful=True
* fit(shuffle=) doesn't relate to RNNs, because to keep inner state consistent withi epoch, you need to set stateful=True and batch_size=1, so there is nothing to shuffle
* https://stackoverflow.com/questions/43882796/when-does-keras-reset-an-lstm-state




In [0]:
batch_size = 100

def train_generator():
  while True:

    #seq_length = np.random.randint(low=2, high=200)
    #x_train, y_train = [], []
    
    cur_pos = np.random.randint(low=0, high=n_chars-batch_size-1)

    x_train = np_utils.to_categorical(num_data[cur_pos:cur_pos+batch_size], num_classes=n_vocab)
    y_train = np_utils.to_categorical(num_data[cur_pos+1:cur_pos+batch_size+1], num_classes=n_vocab)

    yield np.reshape(x_train, (1, batch_size, n_vocab)), np.reshape(y_train, (1, batch_size, n_vocab))


In [110]:
model.fit_generator(train_generator(), steps_per_epoch=100, epochs=200, verbose=1, callbacks=callbacks_list)


Epoch 1/200
100/100 [==============================] - 49s 492ms/step - loss: 3.0976

Epoch 00001: loss improved from inf to 3.09758, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 2/200
100/100 [==============================] - 47s 474ms/step - loss: 2.9366

Epoch 00002: loss improved from 3.09758 to 2.93655, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 3/200
100/100 [==============================] - 47s 475ms/step - loss: 2.6773

Epoch 00003: loss improved from 2.93655 to 2.67734, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 4/200
100/100 [==============================] - 48s 475ms/step - loss: 2.3727

Epoch 00004: loss improved from 2.67734 to 2.37265, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 5/200
100/100 [==============================] - 47s 473ms/step - loss: 2.2552

Epoch 00005: loss improved from 2.37265 to 2.25517

KeyboardInterrupt: ignored

## Create new stateful model with same weights and passing cell state for next prediction
No need to pass all the data from beginning, can feed with 1 example per model.predict, saving state from previous prediction
https://stackoverflow.com/questions/48760472/how-to-use-the-keras-model-to-forecast-for-future-dates-or-events

In [111]:
model2 = Sequential()
model2.add(LSTM(500, batch_input_shape=(1, None, n_vocab), return_sequences=True, stateful=True))
model2.add(LSTM(500, batch_input_shape=(1, None, n_vocab), return_sequences=True, stateful=True))
model2.add(LSTM(500, batch_input_shape=(1, None, n_vocab), return_sequences=False, stateful=True))

model2.add(Dense(n_vocab, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (1, None, 500)            1112000   
_________________________________________________________________
lstm_5 (LSTM)                (1, None, 500)            2002000   
_________________________________________________________________
lstm_6 (LSTM)                (1, 500)                  2002000   
_________________________________________________________________
dense_2 (Dense)              (1, 55)                   27555     
Total params: 5,143,555
Trainable params: 5,143,555
Non-trainable params: 0
_________________________________________________________________


In [0]:
filename = "/content/drive/My Drive/nondualbot/best_current_weights_2.hdf5"
model2.load_weights(filename)
model2.compile(loss='categorical_crossentropy', optimizer='adam')

Predict next letter, save state, predict next letter with saved state

In [0]:
def apply_temp(x, temp=1):
  x = x**temp
  return x/sum(x)

In [114]:
model2.reset_states()

question = 'What is the meaning of life, death and everything?'
question_features = np_utils.to_categorical([char_to_int[char] for char in question], num_classes=n_vocab)
#no need to fill array with zeros
#question_features = np.append(np.zeros(shape=(max(seq_length-question_features.shape[0],0), n_vocab)), question_features, axis=0)

print(question)

#predict question char by char

sys.stdout.write(' ')



sys.stdout.write('\n')

for t in np.arange(0,3.1,0.5):
  
  model2.reset_states()
  
  for question_char in question_features:
    model_response = model2.predict(question_char.reshape(1,1,n_vocab))
  
  print('Temperature: '+str(t))
  
  text_response = ''
  
  for _ in range(150):

    model_input = np.zeros(n_vocab)
    model_input[char_n] = 1
    model_response = model2.predict(model_input.reshape(1,1,n_vocab))[0]
    char_n = np.random.choice(len(model_response), p=apply_temp(model_response, temp=t))
    print_char = int_to_char[char_n]
    text_response += print_char
    
  print(text_response,'\n')

What is the meaning of life, death and everything?
 
Temperature: 0.0
yzHUxRYmGB tcQhVuuRzGKt?WoKMQlCBuNhIdYruunmBWCoODgCDTEKCQ-JKcmeFFvNlePBWrPtMOjFQWmHcWHtMcffuQAxkGl-i-oqDJeHhQcItbnPCJgLYDYTVqkaTLEcTI,ep,bb-MTCNnNU iQ 

Temperature: 0.5
es front? How can youuriezwerely arise fongun. Judo tyming aut.tWi seeias tolds he tany? Dood. Learts heme work ,iresincepons, bo,mvaib, becielts? nis 

Temperature: 1.0
The more being all must be suffering bnyond truth, all. The I am man? You cannot know my attitude and nexarns to save fire, cause -- ir awarenes sappi 

Temperature: 1.5
Tners is the Supreme. What is so more only as a mine. I am not still in the real is in surrender and silence and consciousness. That I am. Whatever th 

Temperature: 2.0
I am consciousness and needs. Another states of the mind is easy. What is beyond the self cannot be a state of experience. You are what is the world t 

Temperature: 2.5
S for the mind and pleasure and seeks for the mind. I am not that I am not that 

#Word level model
Using GLOVE pre-computed word embeddings http://nlp.stanford.edu/data/glove.6B.zip

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

https://stackoverflow.com/questions/42064690/using-pre-trained-word2vec-with-lstm-for-word-generation

https://gist.github.com/maxim5/c35ef2238ae708ccb0e55624e9e0252b


Glove embedding is used only for decoding data into vector space, then lstm trained as usual regression problem. For text generation embedding is used to decode model forecasts back into words (closest to forecasted vector). Temperature can be used as finding n closest words and use distance as probabilities


In [0]:
file = open('/content/drive/My Drive/nondualbot/glove.6B.100d.txt', 'r') 

In [30]:
embeddings_index = {}
for line in file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
file.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
embeddings_len = len(embeddings_index['the'])
embeddings_index['endofsentence'] = np.random.random(embeddings_len)


In [32]:
import string
data_prep = data.replace('.', ' endofsentence ').lower()
table = str.maketrans({key: None for key in string.punctuation})
data_prep = data_prep.translate(table)
data_prep = data_prep.split(' ')
data_prep = ' '.join(data_prep)
print(data_prep[:250])

that which permeates all which nothing transcends and which like the universal space around us fills everything completely from within and without that supreme nondual brahman  that thou art endofsentence  the seeker is he who is in search of himself


In [0]:
data_gloves = []
new_words_counter = 0

for token in data_prep.split(' '):
  glove_vec = embeddings_index.get(token)
  
  if glove_vec is None:
    glove_vec = np.random.random(embeddings_len)
    embeddings_index[token] = glove_vec
    new_words_counter+=1
    #print(token)
  
  data_gloves.append(glove_vec)
  
print(new_words_counter)

In [0]:
n_words = len(data_gloves)

In [0]:
seq_size = 25

def train_generator_words():
  while True:

    #seq_length = np.random.randint(low=2, high=200)
    #x_train, y_train = [], []
    
    cur_pos = np.random.randint(low=0, high=n_words-seq_size-1)

    x_train = data_gloves[cur_pos:cur_pos+seq_size]
    y_train = data_gloves[cur_pos+1:cur_pos+seq_size+1]

    yield np.reshape(x_train, (1, seq_size, embeddings_len)), np.reshape(y_train, (1, seq_size, embeddings_len))


In [46]:


word_model = Sequential()
word_model.add(LSTM(256, input_shape=(None, embeddings_len), return_sequences=True, stateful=False))
#word_model.add(Dropout(0.5))
word_model.add(LSTM(128, input_shape=(None, embeddings_len), return_sequences=True, stateful=False))
#word_model.add(Dropout(0.5))

word_model.add(Dense(embeddings_len, activation='linear'))
word_model.compile(loss='mean_squared_error', optimizer='adam')
word_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, None, 256)         365568    
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 128)         197120    
_________________________________________________________________
dense_3 (Dense)              (None, None, 100)         12900     
Total params: 575,588
Trainable params: 575,588
Non-trainable params: 0
_________________________________________________________________


In [0]:
word_model.fit_generator(train_generator_words(), steps_per_epoch=100, epochs=200, verbose=1)


Epoch 1/200
100/100 [==============================] - 10s 103ms/step - loss: 0.2068
Epoch 2/200
100/100 [==============================] - 9s 90ms/step - loss: 0.1774
Epoch 3/200
100/100 [==============================] - 9s 87ms/step - loss: 0.1669
Epoch 4/200
 91/100 [==========================>...] - ETA: 0s - loss: 0.1716